In [ ]:
# WARNING in this tutorial we are using kfp==2.0.0b13
# you migth also have to run pip install kfp --pre

In [ ]:
import os
from kfp.registry import RegistryClient
from dotenv import load_dotenv

load_dotenv()
bucket = os.getenv("bucket")
gcp_project = os.getenv("gcp_project")
gcp_service_account = os.getenv("gcp_service_account")
kubeflow_pipelines_artifact_registyr = os.getenv('kubeflow_pipelines_artifact_registyr')

client = RegistryClient(host=f"https://europe-west1-kfp.pkg.dev/{gcp_project}/{kubeflow_pipelines_artifact_registyr}")

In [ ]:
client.list_packages()

# Creating a super simple pipeline

However it is a littel diffrent from the one in previous blog post on how arguments are defined and that we compile to a yaml file instead. 

In [ ]:
import kfp.dsl as dsl
import os
from dotenv import load_dotenv
from kfp.v2.dsl import component
from kfp.v2 import compiler
from google.cloud import aiplatform as aip

def train_model(input: float) -> float:
    return 2.0 + input

def ingetst_data(input: float=2) -> float:
    return 2.0

# Create components for the ingestion and training functions
ingest_data_component = component(ingetst_data)
train_component = component(train_model)


# Define the pipeline using the Kubeflow Pipelines SDK
@dsl.pipeline(
    name="ltv-train",
)
def add_pipeline():
    # Instantiate the ingest_data_component and store its output
    ingest_data = ingest_data_component(input=3.0)
    
    # Instantiate the train_component, passing the output from the ingest_data_component
    train_model = train_component(input=ingest_data.output)
    
    # Disable caching for the train_model component to ensure it runs every time
    train_model.set_caching_options(False)

# Compile the pipeline to generate a JSON file for execution
compiler.Compiler().compile(pipeline_func=add_pipeline, package_path="local_run.yaml")

In [ ]:
templateName, versionName = client.upload_pipeline(
  file_name="local_run.yaml",
  tags=["v1", "latest"],
  extra_headers={"description":"This is an example pipeline template."})